In [10]:
import tensorflow as tf
K = tf.keras

with open('./Data/warpeace_input.txt','r') as f:
    data = f.read()

uniq_char = set(data)

idx2char = dict()
char2idx = dict()

for i,x in enumerate(uniq_char):
    idx2char[i] = x
    char2idx[x] = i

print(char2idx)
print(idx2char)

{'l': 0, '8': 1, '0': 2, '¿': 3, 'v': 4, 'm': 5, '\xa0': 6, 'p': 7, 'U': 8, 'A': 9, 'c': 10, 'H': 11, '*': 12, '¤': 13, '»': 14, '4': 15, '"': 16, ':': 17, '5': 18, '6': 19, '\n': 20, "'": 21, 'w': 22, 't': 23, ' ': 24, 'T': 25, 'g': 26, 'S': 27, 's': 28, 'o': 29, 'k': 30, 'C': 31, '.': 32, 'q': 33, 'z': 34, 'P': 35, '=': 36, '-': 37, 'Q': 38, 'i': 39, 'Ã': 40, '©': 41, 'a': 42, 'V': 43, 'h': 44, 'J': 45, ',': 46, 'B': 47, ')': 48, '/': 49, 'G': 50, 'b': 51, ';': 52, 'L': 53, 'K': 54, '(': 55, 'Y': 56, '3': 57, 'M': 58, 'ï': 59, '!': 60, 'F': 61, 'Z': 62, '?': 63, 'ª': 64, 'R': 65, 'd': 66, 'n': 67, 'I': 68, 'O': 69, 'E': 70, '1': 71, 'N': 72, '2': 73, 'r': 74, 'e': 75, 'f': 76, 'W': 77, 'j': 78, 'D': 79, 'X': 80, 'u': 81, 'y': 82, '7': 83, 'x': 84, '9': 85}
{0: 'l', 1: '8', 2: '0', 3: '¿', 4: 'v', 5: 'm', 6: '\xa0', 7: 'p', 8: 'U', 9: 'A', 10: 'c', 11: 'H', 12: '*', 13: '¤', 14: '»', 15: '4', 16: '"', 17: ':', 18: '5', 19: '6', 20: '\n', 21: "'", 22: 'w', 23: 't', 24: ' ', 25: 'T', 26

In [11]:
VOCAB_SIZE = len(char2idx)
print(VOCAB_SIZE)

86


In [12]:
# Hyper-parameter: size of context window.
SEQ_LEN = 32
STEP = 3
input_sequence = []
output_char = []

for start in range(0,len(data)-SEQ_LEN,STEP):
    input_sequence.append(data[start:start+SEQ_LEN])
    output_char.append(data[start+SEQ_LEN])
print('# Of sequences',len(input_sequence))
NB_SEQ = len(input_sequence)

# Of sequences 1065400


In [13]:
import numpy as np
# With the default dtype=float these arrays will cause OOM.
X_one_hot = np.zeros([NB_SEQ,SEQ_LEN,VOCAB_SIZE],dtype=np.int8)
y_one_hot = np.zeros([NB_SEQ,VOCAB_SIZE],dtype=np.int8)

In [14]:
for i in range(NB_SEQ):
    # Iterate over all sequences.
    # Convert each sequence to indices in the vocabulary.
    seq_as_char = input_sequence[i]
    seq_as_idx  = [char2idx[c] for c in seq_as_char]
    # Convert each character in the sequence to one-hot representation.
    for seq_id,char_idx in enumerate(seq_as_idx):
        X_one_hot[i,seq_id,char_idx] = 1
    y_one_hot[i,char2idx[output_char[i]]] = 1

In [15]:
HIDDEN_STATE_SZ = 512
NB_LAYERS = 3

In [24]:

model = K.Sequential()
model.add(K.layers.LSTM(HIDDEN_STATE_SZ, input_shape=(SEQ_LEN, VOCAB_SIZE), return_sequences=True))
for i in range(NB_LAYERS - 2):
    model.add(K.layers.LSTM(HIDDEN_STATE_SZ, return_sequences=True))
model.add(K.layers.LSTM(HIDDEN_STATE_SZ))
model.add(K.layers.Dense(VOCAB_SIZE))
model.add(K.layers.Activation('softmax'))
model.compile(loss="categorical_crossentropy", optimizer="rmsprop")

In [25]:
model.fit(X_one_hot,y_one_hot,batch_size=128)

Epoch 1/1
  11008/1065400 [..............................] - ETA: 24:48:49 - loss: 3.2472

KeyboardInterrupt: 

In [ ]:
# Used from Keras LSTM example:
#https://github.com/keras-team/keras/blob/master/examples/lstm_text_generation.py
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

#https://github.com/keras-team/keras/blob/master/examples/lstm_text_generation.py
def on_epoch_end(epoch, logs):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - SEQ_LEN - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = data[start_index: start_index + SEQ_LEN]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, SEQ_LEN, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char2idx[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = idx2char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
print()

In [ ]:
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)
model.fit(X_one_hot,y_one_hot,batch_size=128, epochs=5, callbacks=[print_callback])